In [1]:
pip install transformers datasets scikit-learn torch

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load dataset
df = pd.read_csv("datasets/cleanedDataSecondModel.csv")

# Rename for consistency (optional)
df = df[['quote', 'category_encoded']]
df.columns = ['text', 'label']

# Split into train and test
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['text'].tolist(), df['label'].tolist(), test_size=0.2, random_state=42
)


In [3]:
from transformers import DistilBertTokenizerFast

# Load tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

# Tokenize text
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=128)


c:\Marsha\College Apps\Python\environments\nlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import torch

class QuoteDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            key: torch.tensor(val[idx])
            for key, val in self.encodings.items()
        } | {'labels': torch.tensor(self.labels[idx])}

train_dataset = QuoteDataset(train_encodings, train_labels)
test_dataset = QuoteDataset(test_encodings, test_labels)


In [5]:
pip install transformers[torch]


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
pip install accelerate>=0.26.0

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

# Define model
num_labels = len(set(df['label']))
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=num_labels)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    save_strategy="no",
    logging_dir="./logs",
    logging_steps=10
)

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# Train!
trainer.train()


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Marsha\College Apps\Python\environments\nlp\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss
1,4.622600,4.625342
2,4.334300,4.411362
3,3.985300,4.197761
4,3.702100,4.111829


TrainOutput(global_step=400, training_loss=4.238929853439331, metrics={'train_runtime': 1416.0339, 'train_samples_per_second': 2.26, 'train_steps_per_second': 0.282, 'total_flos': 87936504384000.0, 'train_loss': 4.238929853439331, 'epoch': 4.0})

In [8]:
import numpy as np
from sklearn.metrics import classification_report

# Make predictions
preds = trainer.predict(test_dataset)
pred_labels = np.argmax(preds.predictions, axis=1)

# Print evaluation
print(classification_report(test_labels, pred_labels))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.00      0.00      0.00         2
           2       0.25      1.00      0.40         1
           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00         3
           5       0.00      0.00      0.00         1
           6       0.20      0.50      0.29         6
           7       0.00      0.00      0.00         1
           8       0.00      0.00      0.00         1
           9       0.50      0.50      0.50         4
          12       0.00      0.00      0.00         1
          14       0.00      0.00      0.00         1
          15       0.00      0.00      0.00         1
          16       0.08      1.00      0.14         1
          17       0.00      0.00      0.00         1
          18       0.00      0.00      0.00         1
          20       0.50      0.50      0.50         4
          21       0.00    

c:\Marsha\College Apps\Python\environments\nlp\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Marsha\College Apps\Python\environments\nlp\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Marsha\College Apps\Python\environments\nlp\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result

In [13]:
def predict_quote_category(text, model, tokenizer):
    # Tokenize the input
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)

    # Make prediction
    outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    predicted_label = torch.argmax(probs, dim=1).item()

    # Get category name from label ID
    return label_mapping[predicted_label], probs.detach().numpy()


In [ ]:
quote = "Believe in yourself and all that you are."
predicted_category, confidence_scores = predict_quote_category(quote, model, tokenizer)

print("Predicted Category:", predicted_category)
print("Confidence Scores:", confidence_scores)


NameError: name 'label_mapping' is not defined